In [ ]:
import numpy as np
import pandas as pd
import re
import torch
import shutil
import html
import string
!pip install demoji
import demoji
import torch.nn as nn
from nltk.stem.wordnet import WordNetLemmatizer
import csv
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 KB 1.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/sydney-machine-learning/COVID19_sentinentanalysis

Cloning into 'COVID19_sentinentanalysis'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 309 (delta 61), reused 134 (delta 52), pack-reused 165
Receiving objects: 100% (309/309), 93.07 MiB | 27.62 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [ ]:
!pip install transformers==4.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 64.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=285fb4b55604894a7472485eb6947256779562d22cedcc27101140bd750fd01b
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


**Indonesia**

In [ ]:
indo = pd.read_csv("/content/drive/MyDrive/Tweets_Indonesia.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
indo = indo.drop(columns=['Unnamed: 0', 'id', 'user_location'])
indo.head()

,created_at,text
0,Wed Dec 08 04:26:24 +0000 2021,@cryptocclub01 @DeepBrainChain COVID 19 has sl...
1,Wed Dec 08 04:28:11 +0000 2021,RT @baemyooji: aespa received '2021 Visionary ...
2,Wed Dec 08 04:28:28 +0000 2021,RT @FaheemYounus: Omicron does not spread thro...
3,Wed Dec 08 04:29:10 +0000 2021,Hopefully im win..still believe @ElrondNetwork...
4,Wed Dec 08 04:29:24 +0000 2021,RT @michaelmina_lab: I can’t believe this isn’...


In [ ]:
indo.isnull().sum()

created_at    0
text          3
dtype: int64

In [ ]:
indo = indo.dropna(axis=0)

In [ ]:
indo.isnull().sum()

created_at    0
text          0
dtype: int64

In [ ]:
def vax(df):
  
  antivaxxer = df[df['text'].str.find('antivaxxer') != -1]
  antivaxxer = antivaxxer.reset_index()

  vaxxer = df[df['text'].str.find('vaxxer') != -1]
  vaxxer = vaxxer.reset_index()
  final = antivaxxer.append(vaxxer)

  vaccine = df[df['text'].str.find('vaccine') != -1]
  vaccine = vaccine.reset_index()
  final= final.append(vaccine)

  Vaccine = df[df['text'].str.find('Vaccine') != -1]
  Vaccine = Vaccine.reset_index()
  final= final.append(Vaccine)

  pfizer = df[df['text'].str.find('pfizer') != -1]
  pfizer = pfizer.reset_index()
  final = final.append(pfizer)

  BioNTech = df[df['text'].str.find('biontech') != -1]
  BioNTech = BioNTech.reset_index()
  final = final.append(BioNTech)

  sinopharm = df[df['text'].str.find('sinopharm') != -1]
  sinopharm = sinopharm.reset_index()
  final = final.append(sinopharm)

  sinovac = df[df['text'].str.find('sinovac') != -1]
  sinovac = sinovac.reset_index()
  final = final.append(sinovac)

  moderna = df[df['text'].str.find('moderna') != -1]
  moderna = moderna.reset_index()
  final = final.append(moderna)

  astra = df[df['text'].str.find('astrazeneca') != -1]
  astra = astra.reset_index()
  final = final.append(astra)

  covaxin = df[df['text'].str.find('covaxin') != -1]
  covaxin = covaxin.reset_index()
  final = final.append(covaxin)

  covi = df[df['text'].str.find('covishield') != -1]
  covi = covi.reset_index()
  final = final.append(covi)

  sput = df[df['text'].str.find('sputnik v') != -1]
  sput = sput.reset_index()
  final = final.append(sput)

  jan = df[df['text'].str.find('janssen') != -1]
  jan = jan.reset_index()
  final = final.append(jan)

  coro = df[df['text'].str.find('coronavac') != -1]
  coro = coro.reset_index()
  final = final.append(coro)

  nova = df[df['text'].str.find('novavax') != -1]
  nova = nova.reset_index()
  final = final.append(nova)

  convi = df[df['text'].str.find('convidecia') != -1]
  convi = convi.reset_index()
  final = final.append(convi)

  sanofi = df[df['text'].str.find('sanofi') != -1]
  sanofi = sanofi.reset_index()
  final = final.append(sanofi)

  return final 

In [ ]:
data = vax(indo)

In [ ]:
data = data.drop(columns=['index'],axis=1)

In [ ]:
data = data.reset_index()

In [ ]:
data = data.drop(columns=['index'],axis=1)

In [ ]:
data

,created_at,text
0,Mon Jan 03 02:36:12 +0000 2022,RT @Drag0nista: Update on antivaxxer flying to...
1,Fri Dec 17 21:11:09 +0000 2021,RT @patriottakes: Donald Trump is enforcing ma...
2,Mon Dec 20 05:55:43 +0000 2021,My perspective: people who take the vaccine ar...
3,Mon Dec 20 10:31:30 +0000 2021,RT @RabbiPoupko: When #antivaxxers tell you th...
4,Fri Sep 10 13:32:32 +0000 2021,"RT @abednego82: No vaccine, no vote. We can't ..."
...,...,...
29214,Sat Dec 25 01:04:06 +0000 2021,Finally all members gotten vaccinated. Althoug...
29215,Tue Feb 23 00:42:18 +0000 2021,RT @sanofi: What is a recombinant protein adju...
29216,Mon Apr 12 07:36:46 +0000 2021,RT @EU_AseanBC: .@sanofipasteur today announce...
29217,Mon Apr 12 07:36:46 +0000 2021,RT @EU_AseanBC: .@sanofipasteur today announce...


In [ ]:
DATA = pd.read_csv('/content/drive/MyDrive/Indonesia.csv')
DATA.head()

,Unnamed: 0,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Surprise,Joking
0,0,rt update on antivaxxer flying to bris from p...,0,0,0,0,0,0,0,0,1,0,1
1,1,rt donald trump is enforcing mask and vaccine...,0,0,0,0,0,0,0,0,1,0,0
2,2,my perspective people who take the vaccine ar...,0,0,0,0,0,0,1,0,0,0,0
3,3,rt when antivaxxers tell you they care about...,0,0,0,0,0,0,1,0,0,0,0
4,4,rt no vaccine no vote we cannot allow antiv...,0,0,0,0,1,0,1,0,0,0,0


In [ ]:
Indo = pd.DataFrame()
Indo['Tweet'] = DATA['Tweet']
Indo['text'] = data['text']
Indo['date'] = data['created_at']
Indo[labels] = DATA[labels]
Indo.head()

,Tweet,text,date,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Surprise,Joking
0,rt update on antivaxxer flying to bris from p...,RT @Drag0nista: Update on antivaxxer flying to...,Mon Jan 03 02:36:12 +0000 2022,0,0,0,0,0,0,0,0,1,0,1
1,rt donald trump is enforcing mask and vaccine...,RT @patriottakes: Donald Trump is enforcing ma...,Fri Dec 17 21:11:09 +0000 2021,0,0,0,0,0,0,0,0,1,0,0
2,my perspective people who take the vaccine ar...,My perspective: people who take the vaccine ar...,Mon Dec 20 05:55:43 +0000 2021,0,0,0,0,0,0,1,0,0,0,0
3,rt when antivaxxers tell you they care about...,RT @RabbiPoupko: When #antivaxxers tell you th...,Mon Dec 20 10:31:30 +0000 2021,0,0,0,0,0,0,1,0,0,0,0
4,rt no vaccine no vote we cannot allow antiv...,"RT @abednego82: No vaccine, no vote. We can't ...",Fri Sep 10 13:32:32 +0000 2021,0,0,0,0,1,0,1,0,0,0,0


In [ ]:
Indo.to_csv('/content/drive/MyDrive/Indo.csv')

In [ ]:
contractionsWithAnotherInvertedComma = { 
"ain’t": "am not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
"couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
"hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "he’s": "he is",
"how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
"i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
"it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us",
"ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
"mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have",
"shan’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
"she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have",
"so’ve": "so have", "so’s": "so is", "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
"there’d’ve": "there would have", "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
"we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have",
"who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
"would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have",
"y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
"you’re": "you are", "you’ve": "you have"
}

In [ ]:
contractions = { 
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not",
"couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
"i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
"shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she shall / she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
"so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
"y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"
}

In [ ]:
class preprocess():
    def __init__(self, df, contractions, otherContractions):
        self.df = df
        self.contractions = contractions
        self.otherContractions = otherContractions
    
    def lower(self, tweet):
        return tweet.lower()
    
    def abbreviate(self, tweet):
        tweet = tweet.split(' ')
        j = 0
        for str_ in tweet:
            fileName = '/content/COVID19_sentinentanalysis/preprocessing/Abbreviations.txt'
            accessMode = 'r'
            with open(fileName, accessMode) as csvfile:
                dataFromFile = csv.reader(csvfile, delimiter = '=')
                str_ = re.sub('[^a-zA-Z0-9-_.]', '', str_)
                for row in dataFromFile:
                    if str_.upper() == row[0]:
                        tweet[j] = row[1]
                csvfile.close()
            j += 1
        return ' '.join(tweet)
    
    def expand(self, tweet):
        for word in tweet.split():
            if word in self.contractions.keys():
                tweet = tweet.replace(word, self.contractions[word])
            elif word in self.otherContractions.keys():
                tweet = tweet.replace(word, self.otherContractions[word])
        return tweet
    
    def emoji2text(self, tweet):
        emojis = demoji.findall(tweet)
        new_tweet = []
        for word in tweet.split():
            if word in emojis.keys():
                tweet = tweet.replace(word, emojis[word])
                new_tweet.append(emojis[word])
            wordmojis = demoji.findall(word)
            for char in word:
                if char in wordmojis.keys():
                    tweet = tweet.replace(word, wordmojis[char])
        
        return tweet

    def remove_hashtags(self, tweet):
        return re.sub(r'\#w+', '', tweet)
    
    def remove_mentions(self, tweet):
        for word in tweet.split():
            if word[0] == '@':
                tweet = tweet.replace(word, '')
        return tweet

    def remove_punctuations(self, tweet):
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')
        return tweet.translate(trantab)

    def remove_url(self, tweet):
        return re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet, flags = re.MULTILINE)    
    
    def preprocess_tweet(self, tweet):
        tweet = self.lower(tweet)
        tweet = self.abbreviate(tweet)
        tweet = self.expand(tweet)
        tweet = self.emoji2text(tweet)
        tweet = self.remove_mentions(tweet)
        tweet = self.remove_url(tweet)
        tweet = self.remove_hashtags(tweet)
        tweet = self.remove_punctuations(tweet)
        
        return tweet

In [ ]:
pp_class = preprocess(data, contractions, contractionsWithAnotherInvertedComma)
data['text'] = data['text'].apply(lambda x : pp_class.preprocess_tweet(x))

In [ ]:
data['text']

0        rt  update on antivaxxer flying to bris from p...
1        rt  donald trump is enforcing mask and vaccine...
2        my perspective  people who take the vaccine ar...
3        rt  when  antivaxxers tell you they care about...
4        rt  no vaccine  no vote  we cannot allow antiv...
                               ...                        
29214    finally all members gotten vaccinated  althoug...
29215    rt  what is a recombinant protein adjuvanted v...
29216    rt    sanofipasteur today announced their €400...
29217    rt    sanofipasteur today announced their €400...
29218    rt    sanofipasteur today announced their €400...
Name: text, Length: 29219, dtype: object

In [ ]:
labels = ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic',
       'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise',
       'Joking']

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
df = pd.DataFrame()
df['Tweet'] = data['text']
values = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] * 29219
df[labels] = values

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
   def __init__(self,df,tokenizer,max_len):
     self.df = df
     self.tokenizer = tokenizer
     self.max_len = max_len
     self.title = self.df['Tweet']
     self.targets = self.df[labels].values

   def __len__(self):
     return len(self.title)

   def __getitem__(self,index):
     title = str(self.title[index])
     title = " ".join(title.split())

     inputs = self.tokenizer.encode_plus(
         title,
         None,
         add_special_tokens=True,
         max_length=self.max_len,
         padding='max_length',
         return_token_type_ids=True,
         truncation=True,
         return_attention_mask=True,
         return_tensors='pt'
     )
     return {
         'input_ids':inputs['input_ids'].flatten(),
         'attention_mask':inputs['attention_mask'].flatten(),
         'token_type_ids':inputs['token_type_ids'].flatten(),
         'targets':torch.FloatTensor(self.targets[index])
     }  

In [ ]:
MAX_LEN=256

In [ ]:
indo_dataset = CustomDataset(df, tokenizer, MAX_LEN)

In [ ]:
indo_data_loader = torch.utils.data.DataLoader(
    indo_dataset,
    shuffle=False,
    batch_size=8,
    num_workers=0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
class BERT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = BertModel.from_pretrained('bert-base-uncased')
        self.layer2 = torch.nn.Dropout(0.3)
        self.layer3 = torch.nn.Linear(768, 11)

    def forward(self, ids, mask, token_type_ids, return_dict = False):
        unw, out_1 = self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[0], self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[1]
        out_2 = self.layer2(out_1)
        out_final = self.layer3(out_2)
        return out_final

model = BERT()
model.to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT(
  (layer1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

In [ ]:
model = torch.load("/content/drive/MyDrive/bertmodel.pth")

In [ ]:
model.eval()
bert_outputs = []
with torch.no_grad():

  for index, batch in enumerate(indo_data_loader):
         input_ids = batch['input_ids'].to(device, dtype=torch.long)
         attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
         token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
         targets =  batch['targets'].to(device, dtype=torch.float)
         outputs = model(input_ids, attention_mask, token_type_ids)
         bert_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

In [ ]:
test_outputs = np.array(bert_outputs)

for i in range(test_outputs.shape[0]):
    for j in range(test_outputs.shape[1]):
        if test_outputs[i][j] >= 0.5: test_outputs[i][j] = 1
        else: test_outputs[i][j] = 0

In [ ]:
len(test_outputs)

29219

In [ ]:
df[labels] = test_outputs.astype(int)

In [ ]:
df['Tweet'][1]

'rt  donald trump is enforcing mask and vaccine mandates at trump tower in nyc \n \nwatch as an antivaxxer enters trump tower to…'

In [ ]:
test_outputs[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [ ]:
df.head()

,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Surprise,Joking
0,rt update on antivaxxer flying to bris from p...,0,0,0,0,0,0,0,0,1,0,1
1,rt donald trump is enforcing mask and vaccine...,0,0,0,0,0,0,0,0,1,0,0
2,my perspective people who take the vaccine ar...,0,0,0,0,0,0,1,0,0,0,0
3,rt when antivaxxers tell you they care about...,0,0,0,0,0,0,1,0,0,0,0
4,rt no vaccine no vote we cannot allow antiv...,0,0,0,0,1,0,1,0,0,0,0


In [ ]:
df.to_csv('/content/drive/MyDrive/Indonesia.csv')